# 08.06 Model with Simpletransformer and BERT

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=D2Twqhll2Tz96VAPREgcPuIOv9e-dTwYCCVNnyxekQQ&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/twHbV2Z6wksYqV9Al9_DeOCqUVku3QGxx0YEJ5ao1SD_A6hIxIbuE7E
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [0]:
!gcloud config set project reddit-master

Updated property [core/project].


In [0]:
!pip install tqdm

In [0]:
!pip install transformers

     |████████████████████████████████| 368kB 2.8MB/s 
     |████████████████████████████████| 860kB 64.5MB/s 
     |████████████████████████████████| 645kB 60.9MB/s 
     |████████████████████████████████| 1.0MB 51.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=5047b2852767af083d5a934c6946377addd4ce550a1bae888487a407e39967a6
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
!pip install tensorboardx

     |████████████████████████████████| 194kB 2.7MB/s 


In [0]:
pip install simpletransformers

     |████████████████████████████████| 71kB 2.3MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=1a0a90bbadc8caf3d9b1edff5e7c693cb3cd3bfabea435e3bbf9c121261aa2f2
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./

Writing setup.sh


In [0]:
# Download APEX, a required module
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 5566 (delta 10), reused 11 (delta 6), pack-reused 5531
Receiving objects: 100% (5566/5566), 13.36 MiB | 7.44 MiB/s, done.
Resolving deltas: 100% (3591/3591), done.
Created temporary directory: /tmp/pip-ephem-wheel-cache-uatcyqe8
Created temporary directory: /tmp/pip-req-tracker-h1w2hnbu
Created requirements tracker '/tmp/pip-req-tracker-h1w2hnbu'
Created temporary directory: /tmp/pip-install-n_k1nw3t
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-v1fhjh2f
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-h1w2hnbu'
    Running setup.py (path:/tmp/pip-req-build-v1fhjh2f/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.3.1
    running egg_info
    creating /tmp/pip-req-build-v1fhjh2f/pip-egg-info

In [0]:
import pandas as pd
import torch
import timeit
import pickle as pkl

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel

In [0]:
!ls

adc.json  apex	sample_data  setup.sh


In [0]:
!gsutil cp gs://reddit_final_results/red_comments_posts.csv .

Copying gs://reddit_final_results/red_comments_posts.csv...
| [1 files][561.1 MiB/561.1 MiB]                                                
Operation completed over 1 objects/561.1 MiB.                                    


In [0]:
red_comments_posts_df = pd.read_csv("red_comments_posts.csv", index_col=[0])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
red_comments_posts_df.head(5)

,subreddit,body
0,funny,Did a hell of a job with the blue duck then if...
1,funny,And the ones who don't will settle because at ...
2,funny,My grand father fished cods like this 50 years...
3,funny,It’s fake yes but how did they do it? The eff...
4,funny,They should have smoke bombs that have bug poi...


In [0]:
red_comments_posts_df["body"].isna().sum()

34

In [0]:
red_comments_posts_df = red_comments_posts_df.dropna()

In [0]:
# Create a column named subreddit_id 
red_comments_posts_df['labels'] = red_comments_posts_df['subreddit']

In [0]:
red_comments_posts_df.head(5)

,subreddit,body,labels
0,funny,Did a hell of a job with the blue duck then if...,funny
1,funny,And the ones who don't will settle because at ...,funny
2,funny,My grand father fished cods like this 50 years...,funny
3,funny,It’s fake yes but how did they do it? The eff...,funny
4,funny,They should have smoke bombs that have bug poi...,funny


In [0]:
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [0]:
# Fit the encoder to the pandas column
le.fit(list(red_comments_posts_df['subreddit']))

LabelEncoder()

In [0]:
# View the labels (if you want)
list(le.classes_)

['Fitness',
 'IAmA',
 'atheism',
 'aww',
 'europe',
 'funny',
 'gaming',
 'movies',
 'nba',
 'politics',
 'science',
 'technology',
 'todayilearned',
 'worldnews']

In [0]:
# Apply the fitted encoder to the pandas column
red_comments_posts_df['labels'] = le.transform(red_comments_posts_df['subreddit'])

In [0]:
red_comments_posts_df.head(5)

,subreddit,body,labels
0,funny,Did a hell of a job with the blue duck then if...,5
1,funny,And the ones who don't will settle because at ...,5
2,funny,My grand father fished cods like this 50 years...,5
3,funny,It’s fake yes but how did they do it? The eff...,5
4,funny,They should have smoke bombs that have bug poi...,5


In [0]:
# Adapting our DataFrame to what is needed from simpletransformers 

reddit_df = red_comments_posts_df.drop(['subreddit'], axis=1)

In [0]:
reddit_df = reddit_df.rename({'body': 'text'}, axis=1)

In [0]:
reddit_df.head()

,text,labels
0,Did a hell of a job with the blue duck then if...,5
1,And the ones who don't will settle because at ...,5
2,My grand father fished cods like this 50 years...,5
3,It’s fake yes but how did they do it? The eff...,5
4,They should have smoke bombs that have bug poi...,5


In [0]:
# Checking if there are NaN values and delete them

reddit_df["text"].isna().sum()

0

In [0]:
# reddit_df = reddit_df.dropna()

In [0]:
train,test = train_test_split(reddit_df, test_size=0.25)

In [0]:
test.head()

,text,labels
1816709,The other recent trial of a similar cancer vac...,10
907662,Ya!!!! Fuck solitaire and minesweeper. Fuck wo...,11
147699,"Losing all your school friends at once, I supp...",5
479700,Wow! I really like this!! Will I be able to ge...,6
762500,I loved the 300 trailer. Came out of nowhere. ...,7


In [0]:
model_st = ClassificationModel('bert', 'bert-base-cased', num_labels=14, args={'reprocess_input_data': True, 'overwrite_output_dir': True})

100%|██████████| 435779157/435779157 [00:36<00:00, 11875070.51B/s]


In [0]:
%%time

model_st.train_model(train)

Converting to features started.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 2.831515

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:91: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 2.664768Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 2.641786Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 2.699026Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 1.449640Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.798024Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 2.047379Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.473246Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 1.871378Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 1.753028Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Running loss: 1.532873Gradient overflow.  Skipping step, loss scaler 0 reducing loss sc

In [0]:
pkl.dump(model_st, "model_st.pkl")

!gsutil cp model_st.pkl gs://reddit_models

In [0]:
!ls

sample_data
